# Sample Linear Regression

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("groupby-test").getOrCreate()
training = spark.read.csv("sample-files/salary-test.csv", header=True, inferSchema=True)

22/01/25 15:10:30 WARN Utils: Your hostname, rich resolves to a loopback address: 127.0.1.1; using 192.168.68.114 instead (on interface wlp0s20f3)
22/01/25 15:10:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/25 15:10:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/25 15:10:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/25 15:10:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/25 15:10:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
training.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [21]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["Age", "Experience"], outputCol="Independent Features")

In [22]:
output = featureassembler.transform(training)

In [23]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|Age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [24]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [25]:
finalized_data = output.select("Independent Features", "Salary")

In [26]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [29]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
regressor = regressor.fit(train_data)

22/01/25 15:55:58 WARN Instrumentation: [8c46c481] regParam is zero, which might cause numerical instability and overfitting.


In [30]:
regressor.coefficients

DenseVector([0.0, 1589.7436])

In [31]:
regressor.intercept

13358.974358973992

In [32]:
pred_results = regressor.evaluate(test_data)

In [33]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16538.461538461514|
|          [24.0,3.0]| 20000|18128.205128205107|
+--------------------+------+------------------+



/home/rich/.pyenv/versions/3.9.1/envs/pyspark-env/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [34]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1666.6666666666897, 2819855.358316973)